In [1]:
cd C:\Users\ab42313\OneDrive\ABZ_PhD\Programming\NCBI

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

handle = open("genBankRecord_545778205.fasta", "rU")
fasta = list(SeqIO.parse(handle, "fasta", generic_dna))

seqs = []
for i in range(0,len(fasta)):
    seqs.append(fasta[i].seq)
    #print fasta[i].seq

genome = seqs[0]

#MiSeq primers (5'-3')
#fw = 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
#rv = 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'

#Primers from wiki, 8F and 1492R
fw = 'AGAGTTTGATCCTGGCTCAG'
rv = 'CGGTTACCTTGTTACGACTT'

In [3]:

def simulate_pcr_run(genome, fw_primer, rv_primer, approximative_gene_lenght = 2600, mismatches = 2):
    '''
    -approximative_gene_lenght: the approximative expected lenght of the gene
    -mismatches: is set on 2 as in an highly stringent PCR for MiSeq sequencing
    '''
    
    
    ### Hamming distance function to calculate mismatches
    
    
    
    def calculate_hamming_distance(sequence1,sequence2): ###If one is primer should be insered as sequence1
    
        sequence1 = sequence1.upper()              ### Differences must be only related to nucleotides
        sequence2 = sequence2.upper()

        if len(sequence1) >= len(sequence2):       ### Necessary not to go out of range
            seq1 =sequence1
            seq2 =sequence2
        elif len(sequence1) < len(sequence2):
            seq2 =sequence1
            seq1 =sequence2

        hamming = 0

        nucleotides = ('A','C','G','T')
        alt_nucleotides = ('W', 'S','M','K','R','Y','B','D','H','V')


        for i in range(0, len(seq2)):

            if seq1[i] in nucleotides:

                if seq1[i] == seq2[i]:
                    hamming += 0
                    
                continue
            
            elif seq1[i] in alt_nucleotides:


                if seq1[i] == 'W':
                    if seq2[i] != 'A' or seq2[i] != 'T':
                        hamming += 1

                if seq1[i] == 'S':
                    if seq2[i] != 'C' or seq2[i] != 'G':
                        hamming += 1

                if seq1[i] == 'M':
                    if seq2[i] != 'A' or seq2[i] != 'C':
                        hamming += 1

                if seq1[i] == 'K':
                    if seq2[i] != 'G' or seq2[i] != 'T':
                        hamming += 1

                if seq1[i] == 'R':
                    if seq2[i] != 'A' or seq2[i] != 'G':
                        hamming += 1

                if seq1[i] == 'Y':
                    if seq2[i] != 'C' or seq2[i] != 'T':
                        hamming += 1

                if seq1[i] == 'B':
                    if seq2[i] != 'C' or seq2[i] != 'G' or seq2[i] != 'T':
                        hamming += 1
                        
                if seq1[i] == 'D':
                    if seq2[i] != 'A' or seq2[i] != 'G' or seq2[i] != 'T':
                        hamming += 1

                if seq1[i] == 'H':
                    if seq2[i] != 'A' or seq2[i] != 'C' or seq2[i] != 'T':
                        hamming += 1

                if seq1[i] == 'V':
                    if seq2[i] != 'A' or seq2[i] != 'C' or seq2[i] != 'G':
                        hamming += 1

                if seq1[i] == 'N' or seq1[i] == '-':
                    if seq2[i] != 'A' or seq2[i] != 'C' or seq2[i] != 'G'or seq2[i] != 'T':
                        continue


        if len(seq1) > len(seq2):                               ### Correcting the Hamming score for missing nucleotides

            hamming += len(seq1) - len(seq2)

        elif len(seq1) < len(seq2):

            hamming += len(seq2) - len(seq1)




        return hamming
    
    
    
    
    ### PCR simulation function:
    
    
    
    #primer_positions = []
    sequences = []
    gene_len = approximative_gene_lenght + 100
    fw = fw_primer
    rv = rv_primer

    positions = range(0,len(genome)-len(fw))
    len(positions)

    for i in positions:
        
        testing = genome[i:i+len(fw)]
        
        if calculate_hamming_distance(fw, testing) <= mismatches:
            
            for m in range(i + len(rv), gene_len):
                
                if calculate_hamming_distance(rv, genome[m : m + len(rv)]) <= mismatches:
                    
                    if len(genome[i:m+len(rv)]) > gene_len-150:         #''Discarting short reads
                        
                        #print i, m, genome[i:m]
                        sequences.append(str(genome[i:m+len(rv)]))
                        #primer_positions.appen+d(i)
                        #primer_positions.append(m+len(rv))
                  
                        break
    
    return sequences
    #return 'Positions: ', primer_positions
    #return sequences

The program apparently works since it selects fragments.

Problems:
    - "amplified" fragments blast result is not informative at all (raises an error...)
    - MiSeq primers don't work.
    - Primer/Probe mismatches have to be set at high values
    
Will check the code again on monday.